In [ ]:
import os
import copy
from coffea import hist
from coffea import util
import numpy as np
import itertools
import pandas as pd

In [ ]:
dir1 = 'CoffeaOutputsForCombine/Coffea_FirstRun/'
dir2 = 'CoffeaOutputsForCombine/Coffea_SecondRun/'
btagDir = 'LooseBTag/'#'LooseBTag/'
yearDir = '2016/'
APVDir = 'noAPV/'
od = ''


if btagDir == '':
    od = '_oldANdisc'

In [ ]:
Unc = '' #_btagUnc_central_method2
TopPt = '' #_TopReweight
UncDir = Unc + '/'
TopPtDir = TopPt + '/'

In [ ]:
JetHT2016_str = 'TTbarRes_0l_JetHT2016_Data'
TTbar_str = 'TTbarRes_0l_UL16postVFP_TTbar'

In [ ]:
JetHT2016_unweighted = util.load(dir1 + 'JetHT/' + btagDir + yearDir + JetHT2016_str + od + '.coffea')
TTbar_unweighted = util.load(dir1 + 'TT/' + btagDir + yearDir + APVDir + TTbar_str + od + '.coffea')

# Cutflow

In [ ]:
outputs_unweighted =     {'JetHT 2016': JetHT2016_unweighted,
                          'TTbar': TTbar_unweighted
                         }

In [ ]:
for name,output in outputs_unweighted.items(): 
    print("-------Unweighted " + name + "--------")
    for i,j in output['cutflow'].items():        
        print( '%20s : %12d' % (i,j) )

In [ ]:
def mkdir_p(mypath):
    '''Creates a directory. equivalent to using mkdir -p on the command line'''

    from errno import EEXIST
    from os import makedirs,path

    try:
        makedirs(mypath)
    except OSError as exc: # Python >2.5
        if exc.errno == EEXIST and path.isdir(mypath):
            pass
        else: raise

In [ ]:
def DoesDirectoryExist(mypath): #extra precaution (Probably overkill...)
    '''Checks to see if Directory exists before running mkdir_p'''
    import os.path
    from os import path
    
    if path.exists(mypath):
        pass
    else:
        mkdir_p(mypath)

In [ ]:
import matplotlib.pyplot as plt
import warnings
import re # regular expressions
warnings.filterwarnings("ignore")

# ---- Reiterate categories ---- #
ttagcats = ["at", "0t", "1t", "2t"]
btagcats = ["0b", "1b", "2b"]
ycats = ['cen', 'fwd']

list_of_cats = [ t+b+y for t,b,y in itertools.product( ttagcats, btagcats, ycats) ]
list_of_bcats = [ b+y for b,y in itertools.product( btagcats, ycats) ]

# ---- List the Histograms Here ---- #
list_of_hists = ('ttbarmass', 'jetpt', 'jeteta', 'jetphi', 'jety', 'jetdy', 'probept', 'probep')

In [ ]:
maindirectory = os.getcwd() 
print(maindirectory)

In [ ]:
stack_ttbar_opts = {'alpha': 0.8, 'edgecolor':(0,0,0,0.3), 'color': 'red'}
stack_background_opts = {'alpha': 0.8, 'edgecolor':(0,0,0,0.3), 'color': 'yellow'}
stack_error_opts = {'label':'Stat. Unc.', 'hatch':'///', 'facecolor':'None', 'edgecolor':(0,0,0,.5), 'linewidth': 0}
data_err_opts = {'linestyle': 'none', 'marker': '.', 'markersize': 10., 'color': 'k', 'elinewidth': 1}

In [ ]:
# ---- Optional to rescale x-axis of mistag rates ---- #
def forward(x):
    return x**(1/2)

def inverse(x):
    return x**2

# Luminosities, Cross Sections & Scale Factors for MC

In [ ]:
Lum2016 = 35920. # pb^-1 from https://twiki.cern.ch/twiki/bin/viewauth/CMS/PdmVAnalysisSummaryTable

t_BR = 0.67
ttbar_BR = 0.4544 #PDG 2019
ttbar_xs = 831.76  #pb  Set this to 1 if not dividing by sumw, and we get a reasonable normalization for ttbar
toptag_kf = 0.70 # k-factor from https://github.com/cmsb2g/B2GTTbar/blob/master/test/MakeMistag_SubtractAndDivideAntiTag_B2G2016.cc#L472

ttbar2016_sf = ttbar_xs*Lum2016*ttbar_BR*toptag_kf/TTbar_unweighted['cutflow']['sumw']

# -- Should be the same given the same number of events in both wgt and unwgt outputs -- #
print('ttbar 2016 scale factor          = ', ttbar2016_sf)

In [ ]:
Nevts2016 = 625502676 # from dasgoclient
Nevts2016_sf = Nevts2016/JetHT2016_unweighted['cutflow']['all events']

print(Nevts2016_sf)

In [ ]:
BlindingData = True # Hide results from 2017 and 2018 data (if data from those years were included in processing)

# Mistag Rates with $t\bar{t}$ Subtraction for SeparateYears

In [ ]:
SaveDirectory = maindirectory + '/' + 'MistagPlots' + '/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
for icat in list_of_cats:
    if 'at' in icat:
        fig, ax = plt.subplots(
            #nrows=2,
            #ncols=1,
            figsize=(7,5),
            #gridspec_kw={"height_ratios": (3, 1)},
            sharex=True
        )
        title = 'mistag ' + icat[2:]
        
        NumeratorTT2016 = TTbar_unweighted['numerator'].integrate('anacat', icat).integrate('dataset', 'UL16postVFP_TTbar')
        DenominatorTT2016 = TTbar_unweighted['denominator'].integrate('anacat', icat).integrate('dataset', 'UL16postVFP_TTbar')
#         NumeratorTT2017 = copy.copy(NumeratorTT2016)
#         DenominatorTT2017 = copy.copy(DenominatorTT2016)
#         NumeratorTT2018 = copy.copy(NumeratorTT2016)
#         DenominatorTT2018 = copy.copy(DenominatorTT2016)
        NumeratorTT2016.scale(-ttbar2016_sf)
        DenominatorTT2016.scale(-ttbar2016_sf)
#         NumeratorTT2017.scale(-ttbar2017_sf)
#         DenominatorTT2017.scale(-ttbar2017_sf)
#         NumeratorTT2018.scale(-ttbar2018_sf)
#         DenominatorTT2018.scale(-ttbar2018_sf)
        
        Numerator2016 = JetHT2016_unweighted['numerator'].integrate('anacat', icat).integrate('dataset', 'JetHT2016_Data')
        Denominator2016 = JetHT2016_unweighted['denominator'].integrate('anacat', icat).integrate('dataset', 'JetHT2016_Data')
        Numerator2016.scale(Nevts2016_sf)
        Denominator2016.scale(Nevts2016_sf)
        Numerator2016.add(NumeratorTT2016)
        Denominator2016.add(DenominatorTT2016)
        
        if not BlindingData:
            Numerator2017 = JetHT2017_unweighted['numerator'].integrate('anacat', icat).integrate('dataset', 'JetHT2017_Data')
            Denominator2017 = JetHT2017_unweighted['denominator'].integrate('anacat', icat).integrate('dataset', 'JetHT2017_Data')
            Numerator2017.scale(Nevts2017_sf)
            Denominator2017.scale(Nevts2017_sf)
            Numerator2017.add(NumeratorTT2017)
            Denominator2017.add(DenominatorTT2017)

            Numerator2018 = JetHT2018_unweighted['numerator'].integrate('anacat', icat).integrate('dataset', 'JetHT2018_Data')
            Denominator2018 = JetHT2018_unweighted['denominator'].integrate('anacat', icat).integrate('dataset', 'JetHT2018_Data')
            Numerator2018.scale(Nevts2018_sf)
            Denominator2018.scale(Nevts2018_sf)
            Numerator2018.add(NumeratorTT2018)
            Denominator2018.add(DenominatorTT2018)
        
        mistag2016 = hist.plotratio(num = Numerator2016, denom = Denominator2016, ax=ax, clear=False,
                                error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                unc = 'num')
        if not BlindingData:
            mistag2017 = hist.plotratio(num = Numerator2017, denom = Denominator2017, ax=ax, clear=False,
                                    error_opts={'marker': 's', 'markersize': 7., 'color': 'g', 'elinewidth': 1},
                                    unc = 'num')
            mistag2018 = hist.plotratio(num = Numerator2018, denom = Denominator2018, ax=ax, clear=False,
                                    error_opts={'marker': '^', 'markersize': 7., 'color': 'b', 'elinewidth': 1},
                                    unc = 'num')

        ax.set_ylabel('Mistag Rates')
        ax.set_title(title)
        leg = ax.legend(labels=["2016", "2017", "2018"])
        
        # ---- Optional x-axis scaling ---- #
        plt.ylim(bottom = 0, top = 1.)
        plt.xlim(left = 100, right = 3000)
        
        # ---- Optional x-axis scaling ---- #
        #ax.set_xscale('function', functions=(forward, inverse))
        #plt.xticks(np.array([0, 500, 1000, 2000, 3000, 4000, 5000]))

        # ---- Optional x-axis scaling ---- #
        #ax.set_xscale('function', functions=(forward, inverse))
        #plt.xticks(np.array([500, 1000, 2000, 5000, 10000]))
        #plt.yticks(np.array([.05, .10, .15]))
        #ax.set_xscale('log')
        
        #filename = 'Mistag_bdisc8484_ttbarSubtraction_SeparateYears_' + icat + '.png'
        #plt.savefig(SaveDirectory+filename, bbox_inches="tight")
        #print('\n' + filename + ' saved')
    else:
        continue

# Mistag Rates with $t\bar{t}$ Subtraction for SeparateYears with Inclusive $y$ Region

In [ ]:
""" ---------- Rapidity Inclusive Mistag Rates ---------- """
SaveDirectory = maindirectory + '/' + 'MistagPlots' + '/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already

for btag in btagcats:
    fig, ax = plt.subplots(
        figsize=(7,5),
        sharex=True
    )
    title = 'Inclusive ' + btag + ' Mistag Rate'
    
    NumeratorTT2016cen = TTbar_unweighted['numerator'].integrate('anacat', 'at'+btag+'cen').integrate('dataset', 'TTbar')
    DenominatorTT2016cen = TTbar_unweighted['denominator'].integrate('anacat', 'at'+btag+'cen').integrate('dataset', 'TTbar')
    NumeratorTT2016fwd = TTbar_unweighted['numerator'].integrate('anacat', 'at'+btag+'fwd').integrate('dataset', 'TTbar')
    DenominatorTT2016fwd = TTbar_unweighted['denominator'].integrate('anacat', 'at'+btag+'fwd').integrate('dataset', 'TTbar')
#     NumeratorTT2017cen = copy.copy(NumeratorTT2016cen)
#     DenominatorTT2017cen = copy.copy(DenominatorTT2016cen)
#     NumeratorTT2017fwd = copy.copy(NumeratorTT2016fwd)
#     DenominatorTT2017fwd = copy.copy(DenominatorTT2016fwd)
#     NumeratorTT2018cen = copy.copy(NumeratorTT2016cen)
#     DenominatorTT2018cen = copy.copy(DenominatorTT2016cen)
#     NumeratorTT2018fwd = copy.copy(NumeratorTT2016fwd)
#     DenominatorTT2018fwd = copy.copy(DenominatorTT2016fwd)
    
    NumeratorTT2016cen.scale(-ttbar2016_sf)
    DenominatorTT2016cen.scale(-ttbar2016_sf)
    NumeratorTT2016fwd.scale(-ttbar2016_sf)
    DenominatorTT2016fwd.scale(-ttbar2016_sf)
#     NumeratorTT2017cen.scale(-ttbar2017_sf)
#     DenominatorTT2017cen.scale(-ttbar2017_sf)
#     NumeratorTT2017fwd.scale(-ttbar2017_sf)
#     DenominatorTT2017fwd.scale(-ttbar2017_sf)
#     NumeratorTT2018cen.scale(-ttbar2018_sf)
#     DenominatorTT2018cen.scale(-ttbar2018_sf)
#     NumeratorTT2018fwd.scale(-ttbar2018_sf)
#     DenominatorTT2018fwd.scale(-ttbar2018_sf)

    
    
    Num_cen2016 = JetHT2016_unweighted['numerator'].integrate('anacat', 'at'+btag+'cen').integrate('dataset', 'JetHT2016_Data')
    Denom_cen2016 = JetHT2016_unweighted['denominator'].integrate('anacat', 'at'+btag+'cen').integrate('dataset', 'JetHT2016_Data')
    Num_cen2016.scale(Nevts2016_sf)
    Denom_cen2016.scale(Nevts2016_sf)
    Num_cen2016.add(NumeratorTT2016cen)
    Denom_cen2016.add(DenominatorTT2016cen)

    Num_fwd2016 = JetHT2016_unweighted['numerator'].integrate('anacat', 'at'+btag+'fwd').integrate('dataset', 'JetHT2016_Data')
    Denom_fwd2016 = JetHT2016_unweighted['denominator'].integrate('anacat', 'at'+btag+'fwd').integrate('dataset', 'JetHT2016_Data')
    Num_fwd2016.scale(Nevts2016_sf)
    Denom_fwd2016.scale(Nevts2016_sf)
    Num_fwd2016.add(NumeratorTT2016fwd)
    Denom_fwd2016.add(DenominatorTT2016fwd)

    Num_inc2016 = Num_cen2016.add(Num_fwd2016)
    Denom_inc2016 = Denom_cen2016.add(Denom_fwd2016)
    mistag_inclusive2016 = hist.plotratio(num = Num_inc2016, denom = Denom_inc2016, ax=ax, clear=False,
                                          error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                          unc = 'num')

    if not BlindingData:
        Num_cen2017 = JetHT2017_unweighted['numerator'].integrate('anacat', 'at'+btag+'cen').integrate('dataset', 'JetHT2017_Data')
        Denom_cen2017 = JetHT2017_unweighted['denominator'].integrate('anacat', 'at'+btag+'cen').integrate('dataset', 'JetHT2017_Data')
        Num_cen2017.scale(Nevts2017_sf)
        Denom_cen2017.scale(Nevts2017_sf)
        Num_cen2017.add(NumeratorTT2017cen)
        Denom_cen2017.add(DenominatorTT2017cen)

        Num_fwd2017 = JetHT2017_unweighted['numerator'].integrate('anacat', 'at'+btag+'fwd').integrate('dataset', 'JetHT2017_Data')
        Denom_fwd2017 = JetHT2017_unweighted['denominator'].integrate('anacat', 'at'+btag+'fwd').integrate('dataset', 'JetHT2017_Data')
        Num_fwd2017.scale(Nevts2017_sf)
        Denom_fwd2017.scale(Nevts2017_sf)
        Num_fwd2017.add(NumeratorTT2017fwd)
        Denom_fwd2017.add(DenominatorTT2017fwd)

        Num_inc2017 = Num_cen2017.add(Num_fwd2017)
        Denom_inc2017 = Denom_cen2017.add(Denom_fwd2017)
        mistag_inclusive2017 = hist.plotratio(num = Num_inc2017, denom = Denom_inc2017, ax=ax, clear=False,
                                              error_opts={'marker': 's', 'markersize': 7., 'color': 'g', 'elinewidth': 1},
                                              unc = 'num')



        Num_cen2018 = JetHT2018_unweighted['numerator'].integrate('anacat', 'at'+btag+'cen').integrate('dataset', 'JetHT2018_Data')
        Denom_cen2018 = JetHT2018_unweighted['denominator'].integrate('anacat', 'at'+btag+'cen').integrate('dataset', 'JetHT2018_Data')
        Num_cen2018.scale(Nevts2018_sf)
        Denom_cen2018.scale(Nevts2018_sf)
        Num_cen2018.add(NumeratorTT2018cen)
        Denom_cen2018.add(DenominatorTT2018cen)

        Num_fwd2018 = JetHT2018_unweighted['numerator'].integrate('anacat', 'at'+btag+'fwd').integrate('dataset', 'JetHT2018_Data')
        Denom_fwd2018 = JetHT2018_unweighted['denominator'].integrate('anacat', 'at'+btag+'fwd').integrate('dataset', 'JetHT2018_Data')
        Num_fwd2018.scale(Nevts2018_sf)
        Denom_fwd2018.scale(Nevts2018_sf)
        Num_fwd2018.add(NumeratorTT2018fwd)
        Denom_fwd2018.add(DenominatorTT2018fwd)

        Num_inc2018 = Num_cen2018.add(Num_fwd2018)
        Denom_inc2018 = Denom_cen2018.add(Denom_fwd2018)
        mistag_inclusive2018 = hist.plotratio(num = Num_inc2018, denom = Denom_inc2018, ax=ax, clear=False,
                                              error_opts={'marker': '^', 'markersize': 7., 'color': 'b', 'elinewidth': 1},
                                              unc = 'num')
        
    plt.ylim(bottom = 0, top = 0.20)
    plt.xlim(left = 100, right = 7000)

    ax.set_ylabel('Mistag Rates')
    ax.set_title(title)
    leg = ax.legend(labels=["2016", "2017", "2018"])

    # ---- Optional x-axis scaling ---- #
    ax.set_xscale('function', functions=(forward, inverse))
    plt.xticks(np.array([0, 500, 1000, 2000, 3000, 4000, 5000]))
    
    # ---- Optional x-axis scaling ---- #
    #ax.set_xscale('function', functions=(forward, inverse))
    #plt.xticks(np.array([500, 1000, 2000, 5000, 10000]))
    #plt.yticks(np.array([.05, .10, .15]))
    #ax.set_xscale('log')
    
    #filename = 'InclusiveMistag_bdisc8484_ttbarSubtraction_SeparateYears_' + btag + '.png'
    #plt.savefig(SaveDirectory+filename, bbox_inches="tight")
    #print('\n' + filename + ' saved')

# Mistag Rates with $t\bar{t}$ Subtraction for 2016 Data with Inclusive $y$ Region Sorted by b-tag Region

In [ ]:
### COPY THIS CELL FOR OTHER YEARS ###
SaveDirectory = maindirectory + '/' + 'MistagPlots' + '/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already

fig, ax = plt.subplots(
    figsize=(7,5),
    sharex=True
)
title = '2016 Inclusive Mistag Rate'

NumeratorTTcen0b = TTbar_unweighted['numerator'].integrate('anacat', 'at0bcen').integrate('dataset', 'UL16postVFP_TTbar')
DenominatorTTcen0b = TTbar_unweighted['denominator'].integrate('anacat', 'at0bcen').integrate('dataset', 'UL16postVFP_TTbar')
NumeratorTTfwd0b = TTbar_unweighted['numerator'].integrate('anacat', 'at0bfwd').integrate('dataset', 'UL16postVFP_TTbar')
DenominatorTTfwd0b = TTbar_unweighted['denominator'].integrate('anacat', 'at0bfwd').integrate('dataset', 'UL16postVFP_TTbar')
NumeratorTTcen1b = TTbar_unweighted['numerator'].integrate('anacat', 'at1bcen').integrate('dataset', 'UL16postVFP_TTbar')
DenominatorTTcen1b = TTbar_unweighted['denominator'].integrate('anacat', 'at1bcen').integrate('dataset', 'UL16postVFP_TTbar')
NumeratorTTfwd1b = TTbar_unweighted['numerator'].integrate('anacat', 'at1bfwd').integrate('dataset', 'UL16postVFP_TTbar')
DenominatorTTfwd1b = TTbar_unweighted['denominator'].integrate('anacat', 'at1bfwd').integrate('dataset', 'UL16postVFP_TTbar')
NumeratorTTcen2b = TTbar_unweighted['numerator'].integrate('anacat', 'at2bcen').integrate('dataset', 'UL16postVFP_TTbar')
DenominatorTTcen2b = TTbar_unweighted['denominator'].integrate('anacat', 'at2bcen').integrate('dataset', 'UL16postVFP_TTbar')
NumeratorTTfwd2b = TTbar_unweighted['numerator'].integrate('anacat', 'at2bfwd').integrate('dataset', 'UL16postVFP_TTbar')
DenominatorTTfwd2b = TTbar_unweighted['denominator'].integrate('anacat', 'at2bfwd').integrate('dataset', 'UL16postVFP_TTbar')

NumeratorTTcen0b.scale(-ttbar2016_sf)
DenominatorTTcen0b.scale(-ttbar2016_sf)
NumeratorTTfwd0b.scale(-ttbar2016_sf)
DenominatorTTfwd0b.scale(-ttbar2016_sf)
NumeratorTTcen1b.scale(-ttbar2016_sf)
DenominatorTTcen1b.scale(-ttbar2016_sf)
NumeratorTTfwd1b.scale(-ttbar2016_sf)
DenominatorTTfwd1b.scale(-ttbar2016_sf)
NumeratorTTcen2b.scale(-ttbar2016_sf)
DenominatorTTcen2b.scale(-ttbar2016_sf)
NumeratorTTfwd2b.scale(-ttbar2016_sf)
DenominatorTTfwd2b.scale(-ttbar2016_sf)
    
    

Num_cen0b = JetHT2016_unweighted['numerator'].integrate('anacat', 'at0bcen').integrate('dataset', 'JetHT2016_Data')
Denom_cen0b = JetHT2016_unweighted['denominator'].integrate('anacat', 'at0bcen').integrate('dataset', 'JetHT2016_Data')
# Num_cen0b.scale(Nevts2016_sf)
# Denom_cen0b.scale(Nevts2016_sf)
# Num_cen0b.add(NumeratorTTcen0b)
# Denom_cen0b.add(DenominatorTTcen0b)

Num_fwd0b = JetHT2016_unweighted['numerator'].integrate('anacat', 'at0bfwd').integrate('dataset', 'JetHT2016_Data')
Denom_fwd0b = JetHT2016_unweighted['denominator'].integrate('anacat', 'at0bfwd').integrate('dataset', 'JetHT2016_Data')
# Num_fwd0b.scale(Nevts2016_sf)
# Denom_fwd0b.scale(Nevts2016_sf)
# Num_fwd0b.add(NumeratorTTfwd0b)
# Denom_fwd0b.add(DenominatorTTfwd0b)

Num_inc0b = Num_cen0b.add(Num_fwd0b)
Denom_inc0b = Denom_cen0b.add(Denom_fwd0b)
mistag_inclusive0b = hist.plotratio(num = Num_inc0b, denom = Denom_inc0b, ax=ax, clear=False,
                                    error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                    unc = 'num')



Num_cen1b = JetHT2016_unweighted['numerator'].integrate('anacat', 'at1bcen').integrate('dataset', 'JetHT2016_Data')
Denom_cen1b = JetHT2016_unweighted['denominator'].integrate('anacat', 'at1bcen').integrate('dataset', 'JetHT2016_Data')
# Num_cen1b.scale(Nevts2016_sf)
# Denom_cen1b.scale(Nevts2016_sf)
# Num_cen1b.add(NumeratorTTcen1b)
# Denom_cen1b.add(DenominatorTTcen1b)

Num_fwd1b = JetHT2016_unweighted['numerator'].integrate('anacat', 'at1bfwd').integrate('dataset', 'JetHT2016_Data')
Denom_fwd1b = JetHT2016_unweighted['denominator'].integrate('anacat', 'at1bfwd').integrate('dataset', 'JetHT2016_Data')
# Num_fwd1b.scale(Nevts2016_sf)
# Denom_fwd1b.scale(Nevts2016_sf)
# Num_fwd1b.add(NumeratorTTfwd1b)
# Denom_fwd1b.add(DenominatorTTfwd1b)

Num_inc1b = Num_cen1b.add(Num_fwd1b)
Denom_inc1b = Denom_cen1b.add(Denom_fwd1b)
mistag_inclusive1b = hist.plotratio(num = Num_inc1b, denom = Denom_inc1b, ax=ax, clear=False,
                                    error_opts={'marker': 's', 'markersize': 7., 'color': 'g', 'elinewidth': 1},
                                    unc = 'num')



Num_cen2b = JetHT2016_unweighted['numerator'].integrate('anacat', 'at2bcen').integrate('dataset', 'JetHT2016_Data')
Denom_cen2b = JetHT2016_unweighted['denominator'].integrate('anacat', 'at2bcen').integrate('dataset', 'JetHT2016_Data')
# Num_cen2b.scale(Nevts2016_sf)
# Denom_cen2b.scale(Nevts2016_sf)
# Num_cen2b.add(NumeratorTTcen2b)
# Denom_cen2b.add(DenominatorTTcen2b)

Num_fwd2b = JetHT2016_unweighted['numerator'].integrate('anacat', 'at2bfwd').integrate('dataset', 'JetHT2016_Data')
Denom_fwd2b = JetHT2016_unweighted['denominator'].integrate('anacat', 'at2bfwd').integrate('dataset', 'JetHT2016_Data')
# Num_fwd2b.scale(Nevts2016_sf)
# Denom_fwd2b.scale(Nevts2016_sf)
# Num_fwd2b.add(NumeratorTTfwd2b)
# Denom_fwd2b.add(DenominatorTTfwd2b)

Num_inc2b = Num_cen2b.add(Num_fwd2b)
Denom_inc2b = Denom_cen2b.add(Denom_fwd2b)
mistag_inclusive2b = hist.plotratio(num = Num_inc2b, denom = Denom_inc2b, ax=ax, clear=False,
                                    error_opts={'marker': '^', 'markersize': 7., 'color': 'b', 'elinewidth': 1},
                                    unc = 'num')
plt.ylim(bottom = 0, top = 0.20)
plt.xlim([400,10000])

ax.set_ylabel('Mistag Rates')
ax.set_title(title)
leg = ax.legend(labels=["0b", "1b", "2b"])

# ---- Optional x-axis scaling ---- #
ax.set_xscale('function', functions=(forward, inverse))
plt.xticks(np.array([500, 1000, 2000, 5000, 10000]))
# plt.yticks(np.array([.05, .10]))
#ax.set_xscale('log')

filename = 'InclusiveMistag_ttbarSubtraction_JetHT2016.png'
# plt.savefig(SaveDirectory+filename, bbox_inches="tight")
# print('\n' + filename + ' saved')

# Mistag Rates with $t\bar{t}$ Subtraction for All Years Combined with Inclusive $y$ Region

In [ ]:
""" ---------- Rapidity Inclusive Mistag Rate According to b tag; ALL YEARS ---------- """
if not BlindingData:
    SaveDirectory = maindirectory + '/' + 'MistagPlots' + '/' # split histograms into subdirectories
    DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already

    fig, ax = plt.subplots(
        figsize=(7,5),
        sharex=True
    )
    title = 'Inclusive Mistag Rate'

    NumeratorTTcen0b = TTbar_unweighted['numerator'].integrate('anacat', 'at0bcen').integrate('dataset', 'TTbar')
    DenominatorTTcen0b = TTbar_unweighted['denominator'].integrate('anacat', 'at0bcen').integrate('dataset', 'TTbar')
    NumeratorTTfwd0b = TTbar_unweighted['numerator'].integrate('anacat', 'at0bfwd').integrate('dataset', 'TTbar')
    DenominatorTTfwd0b = TTbar_unweighted['denominator'].integrate('anacat', 'at0bfwd').integrate('dataset', 'TTbar')
    NumeratorTTcen1b = TTbar_unweighted['numerator'].integrate('anacat', 'at1bcen').integrate('dataset', 'TTbar')
    DenominatorTTcen1b = TTbar_unweighted['denominator'].integrate('anacat', 'at1bcen').integrate('dataset', 'TTbar')
    NumeratorTTfwd1b = TTbar_unweighted['numerator'].integrate('anacat', 'at1bfwd').integrate('dataset', 'TTbar')
    DenominatorTTfwd1b = TTbar_unweighted['denominator'].integrate('anacat', 'at1bfwd').integrate('dataset', 'TTbar')
    NumeratorTTcen2b = TTbar_unweighted['numerator'].integrate('anacat', 'at2bcen').integrate('dataset', 'TTbar')
    DenominatorTTcen2b = TTbar_unweighted['denominator'].integrate('anacat', 'at2bcen').integrate('dataset', 'TTbar')
    NumeratorTTfwd2b = TTbar_unweighted['numerator'].integrate('anacat', 'at2bfwd').integrate('dataset', 'TTbar')
    DenominatorTTfwd2b = TTbar_unweighted['denominator'].integrate('anacat', 'at2bfwd').integrate('dataset', 'TTbar')

    NumeratorTTcen0b.scale(-ttbar_sf)
    DenominatorTTcen0b.scale(-ttbar_sf)
    NumeratorTTfwd0b.scale(-ttbar_sf)
    DenominatorTTfwd0b.scale(-ttbar_sf)
    NumeratorTTcen1b.scale(-ttbar_sf)
    DenominatorTTcen1b.scale(-ttbar_sf)
    NumeratorTTfwd1b.scale(-ttbar_sf)
    DenominatorTTfwd1b.scale(-ttbar_sf)
    NumeratorTTcen2b.scale(-ttbar_sf)
    DenominatorTTcen2b.scale(-ttbar_sf)
    NumeratorTTfwd2b.scale(-ttbar_sf)
    DenominatorTTfwd2b.scale(-ttbar_sf)



    Num_cen0b = JetHT_unweighted['numerator'].integrate('anacat', 'at0bcen').integrate('dataset', 'JetHT')
    Denom_cen0b = JetHT_unweighted['denominator'].integrate('anacat', 'at0bcen').integrate('dataset', 'JetHT')
    Num_cen0b.scale(Nevts_sf)
    Denom_cen0b.scale(Nevts_sf)
    Num_cen0b.add(NumeratorTTcen0b)
    Denom_cen0b.add(DenominatorTTcen0b)

    Num_fwd0b = JetHT_unweighted['numerator'].integrate('anacat', 'at0bfwd').integrate('dataset', 'JetHT')
    Denom_fwd0b = JetHT_unweighted['denominator'].integrate('anacat', 'at0bfwd').integrate('dataset', 'JetHT')
    Num_fwd0b.scale(Nevts_sf)
    Denom_fwd0b.scale(Nevts_sf)
    Num_fwd0b.add(NumeratorTTfwd0b)
    Denom_fwd0b.add(DenominatorTTfwd0b)

    Num_inc0b = Num_cen0b.add(Num_fwd0b)
    Denom_inc0b = Denom_cen0b.add(Denom_fwd0b)
    mistag_inclusive0b = hist.plotratio(num = Num_inc0b, denom = Denom_inc0b, ax=ax, clear=False,
                                        error_opts={'marker': '.', 'markersize': 13., 'color': 'r', 'elinewidth': 1},
                                        unc = 'num')



    Num_cen1b = JetHT_unweighted['numerator'].integrate('anacat', 'at1bcen').integrate('dataset', 'JetHT')
    Denom_cen1b = JetHT_unweighted['denominator'].integrate('anacat', 'at1bcen').integrate('dataset', 'JetHT')
    Num_cen1b.scale(Nevts_sf)
    Denom_cen1b.scale(Nevts_sf)
    Num_cen1b.add(NumeratorTTcen1b)
    Denom_cen1b.add(DenominatorTTcen1b)

    Num_fwd1b = JetHT_unweighted['numerator'].integrate('anacat', 'at1bfwd').integrate('dataset', 'JetHT')
    Denom_fwd1b = JetHT_unweighted['denominator'].integrate('anacat', 'at1bfwd').integrate('dataset', 'JetHT')
    Num_fwd1b.scale(Nevts_sf)
    Denom_fwd1b.scale(Nevts_sf)
    Num_fwd1b.add(NumeratorTTfwd1b)
    Denom_fwd1b.add(DenominatorTTfwd1b)

    Num_inc1b = Num_cen1b.add(Num_fwd1b)
    Denom_inc1b = Denom_cen1b.add(Denom_fwd1b)
    mistag_inclusive1b = hist.plotratio(num = Num_inc1b, denom = Denom_inc1b, ax=ax, clear=False,
                                        error_opts={'marker': 's', 'markersize': 7., 'color': 'g', 'elinewidth': 1},
                                        unc = 'num')



    Num_cen2b = JetHT_unweighted['numerator'].integrate('anacat', 'at2bcen').integrate('dataset', 'JetHT')
    Denom_cen2b = JetHT_unweighted['denominator'].integrate('anacat', 'at2bcen').integrate('dataset', 'JetHT')
    Num_cen2b.scale(Nevts_sf)
    Denom_cen2b.scale(Nevts_sf)
    Num_cen2b.add(NumeratorTTcen2b)
    Denom_cen2b.add(DenominatorTTcen2b)

    Num_fwd2b = JetHT_unweighted['numerator'].integrate('anacat', 'at2bfwd').integrate('dataset', 'JetHT')
    Denom_fwd2b = JetHT_unweighted['denominator'].integrate('anacat', 'at2bfwd').integrate('dataset', 'JetHT')
    Num_fwd2b.scale(Nevts_sf)
    Denom_fwd2b.scale(Nevts_sf)
    Num_fwd2b.add(NumeratorTTfwd2b)
    Denom_fwd2b.add(DenominatorTTfwd2b)

    Num_inc2b = Num_cen2b.add(Num_fwd2b)
    Denom_inc2b = Denom_cen2b.add(Denom_fwd2b)
    mistag_inclusive2b = hist.plotratio(num = Num_inc2b, denom = Denom_inc2b, ax=ax, clear=False,
                                        error_opts={'marker': '^', 'markersize': 7., 'color': 'b', 'elinewidth': 1},
                                        unc = 'num')
    plt.ylim(bottom = 0, top = 0.15)
    plt.xlim([400,10000])

    ax.set_ylabel('Mistag Rates')
    ax.set_title(title)
    leg = ax.legend(labels=["0b", "1b", "2b"])

    # ---- Optional x-axis scaling ---- #
    ax.set_xscale('function', functions=(forward, inverse))
    plt.xticks(np.array([500, 1000, 2000, 5000, 10000]))
    plt.yticks(np.array([.05, .10, .15]))
    #ax.set_xscale('log')

    #filename = 'InclusiveMistag_bdisc8484_ttbarSubtraction_AllYears.png'
    #plt.savefig(SaveDirectory+filename, bbox_inches="tight")
    #print('\n' + filename + ' saved')